In [2]:
import csv

from astropy.io import ascii
from astropy.coordinates import SkyCoord, Angle
from astropy.io import fits, votable
import numpy as np
from astropy import units as u
from astropy.table import Table

## Prepare source cutout run

This notebook sets up the parameters for bulk source cutouts.

It will create the following files which can be used as input by askap_cutout_daemon.py to produce fits cutouts around the target sources:
* sb10941_srcs_image_params.vot
* targets_10941.csv

It takes the beam listings as input, e.g. beam_listing_SB10941.csv as well as the Selavy continuum source catalogue for the field. The two output files contain the selected source details (name, location) and the beams that are close to the source and should be used to produce the cutout cubelet. The names of the output files are based on the input ASKAP sbid.

Note this code assumes the use of a 36 beam pattern with 3 interleaves, so a total of 108 beams. It also assumes only one set of beams. If multiple observations are to be included, then the measurements sets for all observations must be concatenated so there is a single measurement set per beam.

### Cutout process

askap_cutout_daemon.py should be run on the login node of a cluster (e.g. avatar). It will manage a series of jobs, one for each source, keeping a defined number running at a time. Each job will use CASA to produce a cubelet around the source based on the beams that are close enough to the source to be provide data for the source.

Each job runs make_askap_abs_cutout.sh to start CASA to run the sub_cube_abs.py script. This has the CASA commands to produce the cubelet. It will create a cubelet covering just the immediate region f the source, it will exclude short baselines, to limit the large scale emission included, and will not do any cleaning.

### Input data

The beam listings are produced by running a list_beams script (e.g. list_beams_10941.py) in CASA which extracts the beam positions from each beam measurement set. This assumes the measurement sets have been recentred from their common position output by ASKAPSoft.

The catalogue is the continuum catalogue produced with Selavy as part of the ASKAPSoft pipeline processing.

### Output data

The votable (e.g. sb10941_srcs_image_params.vot) will list the combinations of the target and the beams to include. Each target will thus likely have multiple rows.

The csv (e.g. targets_10941.csv ) lists a single row for each target and includes a variable number of columns at the end, one for each beam to be included when processing that target.

In [24]:
# Parameters for the run
sbid = 14211
catalogue = 'selavy-image.i.SB14211.cont.restored.components.xml'
beam_listing = 'sb{0}/beam_listing_SB{0}.csv'.format(sbid)
flux_peak_min = 15 # mJy/beam
max_sep_close = 0.55*u.deg # First pass maximum distance a beam is allowed to be from the target to be used.
max_sep_far = 0.8*u.deg # Fallback maximum distance a beam is allowed to be from the target to be used if none close

In [19]:
def rename_columns(table):
    names = np.asarray(table.colnames)
    for name in names:
        if name.startswith('col_'):
            table.rename_column(name, name[4:])

def slice_strings(a,start,end):
    if end is None:
        if start > 0:
            raise('end must be present when start is positive')
        b = a.view((str,1)).reshape(len(a),-1)[:,start:]
        return np.frombuffer(b.tostring(),dtype=(str,start*-1))
    
    b = a.view((str,1)).reshape(len(a),-1)[:,start:end]
    return np.frombuffer(b.tostring(),dtype=(str,end-start))


In [16]:
# Read and filter catalogue
src_votable = votable.parse(catalogue, pedantic=False)
table = src_votable.get_first_table().to_table()
rename_columns(table)
targets = table[table['flux_peak']>flux_peak_min]
targets

island_id,component_id,component_name,ra_hms_cont,dec_dms_cont,ra_deg_cont,dec_deg_cont,ra_err,dec_err,freq,flux_peak,flux_peak_err,flux_int,flux_int_err,maj_axis,min_axis,pos_ang,maj_axis_err,min_axis_err,pos_ang_err,maj_axis_deconv,min_axis_deconv,pos_ang_deconv,maj_axis_deconv_err,min_axis_deconv_err,pos_ang_deconv_err,chi_squared_fit,rms_fit_gauss,spectral_index,spectral_curvature,spectral_index_err,spectral_curvature_err,rms_image,has_siblings,fit_is_estimate,spectral_index_from_TT,flag_c4,comment
--,--,,,,deg,deg,arcsec,arcsec,MHz,mJy/beam,mJy/beam,mJy,mJy,arcsec,arcsec,deg,arcsec,arcsec,deg,arcsec,arcsec,deg,arcsec,arcsec,deg,--,mJy/beam,--,--,--,--,mJy/beam,,,,,
bytes20,bytes24,bytes26,bytes12,bytes13,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,int32,bytes100
SB14211_island_1,SB14211_component_1a,J022008-702227,02:20:08.2,-70:22:27,35.033977,-70.374382,0.01,0.01,1419.5,1434.553,4.344,1886.528,11.491,9.16,7.88,74.92,0.03,0.04,0.81,4.78,3.55,63.55,0.09,0.22,1.83,16465.145,11617.237,-99.00,-99.00,0.00,0.00,0.824,0,0,1,1,
SB14211_island_2,SB14211_component_2a,J011816-695147,01:18:16.0,-69:51:47,19.566508,-69.863210,0.00,0.00,1419.5,443.184,0.320,527.888,0.916,8.42,7.77,85.44,0.01,0.01,0.43,3.40,3.02,-13.64,0.16,0.23,1.95,430.546,1779.263,-99.00,-99.00,0.00,0.00,0.365,1,0,1,0,
SB14211_island_2,SB14211_component_2b,J011817-695141,01:18:17.8,-69:51:41,19.574036,-69.861617,0.04,0.04,1419.5,34.450,0.328,39.483,0.918,8.29,7.59,86.87,0.10,0.15,5.37,3.01,2.58,-21.23,2.74,4.14,27.84,430.546,1779.263,-99.00,-99.00,0.00,0.00,0.365,1,0,1,0,
SB14211_island_3,SB14211_component_3a,J020029-723408,02:00:29.6,-72:34:08,30.123284,-72.569137,0.00,0.00,1419.5,399.577,0.476,491.022,1.207,8.38,8.05,59.05,0.01,0.02,1.22,4.17,2.65,5.28,0.01,0.07,0.85,8682.769,7436.683,-99.00,-99.00,0.00,0.00,0.137,0,0,1,1,
SB14211_island_4,SB14211_component_4a,J014154-694126,01:41:54.5,-69:41:26,25.477274,-69.690569,0.03,0.02,1419.5,365.213,8.763,442.708,10.861,8.51,7.83,78.14,0.02,0.04,1.47,3.61,3.16,16.42,0.38,0.57,6.38,1534.939,2425.076,-99.00,-99.00,0.00,0.00,0.144,1,0,1,0,
SB14211_island_4,SB14211_component_4b,J014156-694140,01:41:56.0,-69:41:40,25.483292,-69.694492,0.03,0.02,1419.5,263.033,6.400,314.081,7.763,8.60,7.62,82.03,0.02,0.03,0.63,3.50,3.05,74.31,0.42,0.61,3.60,1534.939,2425.076,-99.00,-99.00,0.00,0.00,0.144,1,0,1,0,
SB14211_island_4,SB14211_component_4c,J014155-694137,01:41:55.5,-69:41:37,25.481120,-69.693621,0.27,0.45,1419.5,37.646,4.734,59.997,7.845,9.88,8.86,0.99,0.27,0.32,11.76,6.98,4.10,-1.94,0.13,0.11,7.05,1534.939,2425.076,-99.00,-99.00,0.00,0.00,0.144,1,0,1,0,
SB14211_island_4,SB14211_component_4d,J014154-694125,01:41:54.2,-69:41:25,25.475695,-69.690419,0.38,0.15,1419.5,25.984,8.843,34.855,12.137,10.45,7.05,56.18,0.42,0.53,2.35,7.12,0.00,50.37,0.06,0.00,2.86,1534.939,2425.076,-99.00,-99.00,0.00,0.00,0.144,1,0,1,0,


In [20]:
# Build the list of beam locations
beams = ascii.read(beam_listing, format='no_header', guess=False, delimiter=',')
names = ('col1','col2', 'col3', 'col4')
new_names = ('filename','interleave', 'ra_rad', 'dec_rad')
beams.rename_columns(names, new_names)

beam_id = slice_strings(beams['filename'], -8, -6)
interleave_id = slice_strings(beams['interleave'], -1, None)# beams['interleave'][:][-1:]
file_interleave = slice_strings(beams['filename'], -14, -13)

ids = np.stack((beam_id, interleave_id), axis=-1)
unique_ids, unique_idx = np.unique(ids, axis=0, return_index=True)

beams['beam_id'] = beam_id
beams['interleave'] = interleave_id
beams['file_interleave'] = file_interleave
unique_beams = beams[beams['interleave'] == beams['file_interleave']]
print (unique_beams[0])

u_beam_locs = SkyCoord(ra=unique_beams['ra_rad']*u.rad, dec=unique_beams['dec_rad']*u.rad, frame='icrs')
u_beam_locs


                                                        filename                                                        interleave     ra_rad        dec_rad     beam_id file_interleave
----------------------------------------------------------------------------------------------------------------------- ---------- -------------- -------------- ------- ---------------
/avatar/lynnca/ASKAP/SMC/data/pilot_obs/ms_data/14211/GASKAP_M344-06A/scienceData_SB14211_GASKAP_M344-06_A.beam00_SL.ms          A 0.389530257754 -1.18183047316      00               A


<SkyCoord (ICRS): (ra, dec) in deg
    [(22.31843976, -67.71389821), (21.73148601, -68.58754846),
     (21.09605949, -69.45848946), (20.40573324, -70.32662931),
     (19.65292792, -71.19181737), (18.82865009, -72.05383086),
     (24.07884002, -68.3282884 ), (23.54549759, -69.20764739),
     (22.96663009, -70.08488367), (22.33591791, -70.95995901),
     (21.64584458, -71.83278443), (20.88740641, -72.70320794),
     (26.36959446, -68.03625329), (25.93475059, -68.92255347),
     (25.46350324, -69.80725335), (24.95079679, -70.69040531),
     (24.39061172, -71.57202837), (23.77572672, -72.45210037),
     (28.25957442, -68.60450791), (27.89130049, -69.49514298),
     (27.4915095 , -70.38467465), (27.05564123, -71.27320471),
     (26.57822433, -72.16081048), (26.05263861, -73.04753892),
     (30.51673508, -68.25485512), (30.2461627 , -69.15001405),
     (29.9534192 , -70.04435848), (29.63531985, -70.93804939),
     (29.28804785, -71.83123512), (28.90698957, -72.72404865),
     (32.5253176 , -

In [21]:
def get_beams_near_src(target_loc, beam_locs, beams, max_sep = 0.8*(1*u.deg)):
    """
    Find the beams within a certain angular distance of the target location.
    
    Parameters
    ----------
    target_loc: SkyCoord
        The sky location of the target.
    beam_locs: SkyCoord[]
        The locations of each beam.
    beams: str[]
        Array of beam names
    max_sep: dimension
        Maximum distance a beam centre can be from the target to be included (optional).
    
    Returns
    -------
    List of beam names within the requested distance and a list of their distances from the target.
    """
    beam_sep = beam_locs.separation(target_loc)
    beams_covering_target = beam_sep < max_sep
    src_beams = beams[beams_covering_target]
    src_beam_sep = beam_sep[beams_covering_target]
    return src_beams, src_beam_sep



In [26]:
comp_names = []
comp_ra = []
comp_dec = []
included_beam_nums = []
included_beam_interleaves = []
included_beam_ids = []
included_beam_sep = []

for tgt in targets:
    target_loc = SkyCoord(ra=tgt['ra_deg_cont']*u.degree, dec=tgt['dec_deg_cont']*u.degree, frame='icrs')
    src_beams, src_beam_sep = get_beams_near_src(target_loc, u_beam_locs, unique_beams, max_sep=max_sep_close)
    if len(src_beams) == 0:
        print ("No beams close to {}, checking out to {}".format(tgt['component_name'], max_sep_far))
        src_beams, src_beam_sep = get_beams_near_src(target_loc, u_beam_locs, unique_beams, max_sep=max_sep_far)

    for i in range(len(src_beams)):
        comp_names.append(tgt['component_name'])
        comp_ra.append(tgt['ra_deg_cont'])
        comp_dec.append(tgt['dec_deg_cont'])
        included_beam_nums.append(src_beams['beam_id'].data[i])
        included_beam_interleaves.append(src_beams['interleave'].data[i])
        included_beam_ids.append(src_beams['beam_id'].data[i]+src_beams['interleave'].data[i])
        included_beam_sep.append(src_beam_sep.to(u.deg).value[i])

image_params = Table()
image_params['component_name'] = comp_names
image_params['comp_ra'] = comp_ra
image_params['comp_dec'] = comp_dec
image_params['beam_nums'] = included_beam_nums
image_params['beam_interleaves'] = included_beam_interleaves
image_params['beam_ids'] = included_beam_ids
image_params['beam_sep'] = included_beam_sep

image_params_vot = votable.from_table(image_params)
filename = "sb{}_srcs_image_params.vot".format(sbid)
votable.writeto(image_params_vot, filename)

print ("Produced VO table file {} with the following contents:".format(filename))

image_params_vot.get_first_table()


No beams close to J022008-702227, checking out to 0.8 deg
No beams close to J011432-732143, checking out to 0.8 deg
No beams close to J022100-710732, checking out to 0.8 deg
No beams close to J012325-672312, checking out to 0.8 deg
No beams close to J015136-735424, checking out to 0.8 deg
No beams close to J010930-713456, checking out to 0.8 deg
No beams close to J010932-713452, checking out to 0.8 deg
No beams close to J011808-694601, checking out to 0.8 deg
No beams close to J011808-694558, checking out to 0.8 deg
No beams close to J022126-712319, checking out to 0.8 deg
No beams close to J022126-712315, checking out to 0.8 deg
No beams close to J011357-702531, checking out to 0.8 deg
No beams close to J014114-740732, checking out to 0.8 deg
No beams close to J022139-692954, checking out to 0.8 deg
No beams close to J012350-735041, checking out to 0.8 deg
No beams close to J012348-735033, checking out to 0.8 deg
No beams close to J014036-673945, checking out to 0.8 deg
No beams close

<Table masked=True length=1109>
component_name  comp_ra   comp_dec  ... beam_ids       beam_sep     
    str14       float64   float64   ...   str3         float64      
-------------- --------- ---------- ... -------- -------------------
J022008-702227 35.033977 -70.374382 ...      31B  0.6910335763273346
J022008-702227 35.033977 -70.374382 ...      32B  0.5964884734312271
J011816-695147 19.566508  -69.86321 ...      03A   0.544411936855618
J011816-695147 19.566508  -69.86321 ...      02C  0.5379072930573466
J011817-695141 19.574036 -69.861617 ...      03A   0.544436964486441
J011817-695141 19.574036 -69.861617 ...      02C  0.5357452862337521
J020029-723408 30.123284 -72.569137 ...      29A  0.3944594201508801
J020029-723408 30.123284 -72.569137 ...      34A  0.5055389851559561
J020029-723408 30.123284 -72.569137 ...      29B 0.36940897047780946
J020029-723408 30.123284 -72.569137 ...      28C  0.2880118803162589
           ...       ...        ... ...      ...                 ...
J0

In [27]:
print ("Count of the number of sources using each beam")
ar = np.array(included_beam_ids)
for i in range(36):
    for interleave in ('A', 'B', 'C'):
        key = '{:02d}{}'.format(i, interleave)
        count = len(ar[ar==key])
        print (key, count)
print ('mean targets per beam {:.2f}'.format(len(ar)/(36*3)))
mean_beams_per_source = len(ar) / len(targets)
print ('mean beams per target {:.2f}'.format(mean_beams_per_source))

Count of the number of sources using each beam
00A 6
00B 6
00C 8
01A 5
01B 9
01C 12
02A 21
02B 16
02C 24
03A 15
03B 9
03C 11
04A 7
04B 8
04C 7
05A 6
05B 3
05C 11
06A 9
06B 10
06C 11
07A 15
07B 18
07C 15
08A 11
08B 7
08C 13
09A 8
09B 8
09C 8
10A 9
10B 5
10C 6
11A 3
11B 10
11C 20
12A 14
12B 19
12C 16
13A 11
13B 11
13C 13
14A 12
14B 10
14C 9
15A 11
15B 6
15C 8
16A 5
16B 2
16C 10
17A 8
17B 12
17C 11
18A 14
18B 7
18C 8
19A 11
19B 13
19C 16
20A 10
20B 10
20C 6
21A 6
21B 5
21C 5
22A 8
22B 10
22C 17
23A 13
23B 16
23C 8
24A 16
24B 17
24C 10
25A 8
25B 8
25C 12
26A 11
26B 11
26C 4
27A 5
27B 8
27C 7
28A 8
28B 6
28C 8
29A 12
29B 10
29C 13
30A 10
30B 13
30C 9
31A 10
31B 13
31C 12
32A 8
32B 10
32C 10
33A 12
33B 19
33C 14
34A 6
34B 10
34C 6
35A 9
35B 12
35C 11
mean targets per beam 10.27
mean beams per target 2.99


In [28]:
ar = np.array(included_beam_ids)
print ('Count of total beam usage={}\nNumber using beam 02A={}'.format(len(ar), len(ar[ar=='02A'])))

Count of total beam usage=1109
Number using beam 02A=21


In [29]:
print ('Example source and included beams')
tgt = targets[100]
target_loc = SkyCoord(ra=tgt['ra_deg_cont']*u.degree, dec=tgt['dec_deg_cont']*u.degree, frame='icrs')
src_beams, src_beam_sep = get_beams_near_src(target_loc, u_beam_locs, unique_beams)
print(tgt['component_name'])
print(tgt['ra_deg_cont'])
print(tgt['dec_deg_cont'])
print(src_beams['beam_id'].data)

Example source and included beams
J015627-694230
29.113382
-69.708402
['19' '25' '26' '19' '18' '19' '25']


In [30]:
# Create the csv file of targets and included beams
csv_filename = 'targets_{}.csv'.format(sbid)
with open(csv_filename, 'w') as csvfile:
    tgtwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    tgtwriter.writerow(['index', 'component_name', 'ra', 'dec', 'beams'])
    i = 1;
    for tgt in targets:
        comp_name = tgt['component_name']
        row = []
        row.append(str(i))
        row.append(comp_name)
        row.append(tgt['ra_deg_cont'])
        row.append(tgt['dec_deg_cont'])
        for tgt_beam in image_params:
            if comp_name == tgt_beam['component_name']:
                row.append(tgt_beam['beam_ids'])
        tgtwriter.writerow(row)
        i+= 1

print ("Produced VO table file {} with {} target beam combos.".format(filename, len(image_params_vot.get_first_table().array)))
print ('Produced csv file {} with {} targets (mean {:.2f} beams/target).'.format(csv_filename, i, mean_beams_per_source))

Produced VO table file sb14211_srcs_image_params.vot with 1109 target beam combos.
Produced csv file targets_14211.csv with 372 targets (mean 2.99 beams/target).
